Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [53]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [54]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f, encoding='latin1')
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [55]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [56]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))/ predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [12]:
batch_size = 128
number_of_hidden_nodes = 1024
gamma = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights_input_to_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, number_of_hidden_nodes]))
    biases_hidden = tf.Variable(tf.zeros([number_of_hidden_nodes]))
    weights_hidden_to_softmax = tf.Variable(tf.truncated_normal([number_of_hidden_nodes, num_labels]))
    biases_softmax = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    relu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input_to_hidden) + biases_hidden)
    logits = tf.matmul(relu,weights_hidden_to_softmax) + biases_softmax
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+\
                            gamma*tf.reduce_mean(tf.nn.l2_loss(weights_input_to_hidden))+\
                            gamma*tf.reduce_mean(tf.nn.l2_loss(weights_hidden_to_softmax))
    
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input_to_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu,weights_hidden_to_softmax) + biases_softmax)
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input_to_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu,weights_hidden_to_softmax) + biases_softmax)

In [13]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 679.656433
Minibatch accuracy: 7.0%
Validation accuracy: 33.9%
Minibatch loss at step 500: 196.350632
Minibatch accuracy: 82.0%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 116.235764
Minibatch accuracy: 78.1%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 68.283997
Minibatch accuracy: 89.8%
Validation accuracy: 83.3%
Minibatch loss at step 2000: 41.223690
Minibatch accuracy: 89.1%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 25.240128
Minibatch accuracy: 87.5%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 15.510824
Minibatch accuracy: 85.9%
Validation accuracy: 86.8%
Test accuracy: 93.2%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [15]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 1000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [17]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 17.315144
Training accuracy: 8.4%
Validation accuracy: 11.8%
Loss at step 100: 1.585612
Training accuracy: 78.0%
Validation accuracy: 65.9%
Loss at step 200: 0.892909
Training accuracy: 84.1%
Validation accuracy: 67.2%
Loss at step 300: 0.560966
Training accuracy: 89.5%
Validation accuracy: 67.4%
Loss at step 400: 0.365415
Training accuracy: 92.8%
Validation accuracy: 67.6%
Loss at step 500: 0.242488
Training accuracy: 95.0%
Validation accuracy: 68.0%
Loss at step 600: 0.166528
Training accuracy: 96.7%
Validation accuracy: 68.2%
Loss at step 700: 0.120399
Training accuracy: 98.3%
Validation accuracy: 68.5%
Loss at step 800: 0.091795
Training accuracy: 98.9%
Validation accuracy: 68.8%
Loss at step 900: 0.072936
Training accuracy: 99.2%
Validation accuracy: 69.1%
Loss at step 1000: 0.060273
Training accuracy: 99.4%
Validation accuracy: 69.3%
Loss at step 1100: 0.051220
Training accuracy: 99.6%
Validation accuracy: 69.5%
Loss at step 1200: 0.044334
Training acc

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [21]:
batch_size = 128
number_of_hidden_nodes = 1024
gamma = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    keep_prob = tf.placeholder(tf.float32)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
  
    # Variables.
    weights_input_to_hidden = tf.Variable(tf.truncated_normal([image_size * image_size, number_of_hidden_nodes]))
    biases_hidden = tf.Variable(tf.zeros([number_of_hidden_nodes]))
    weights_hidden_to_softmax = tf.Variable(tf.truncated_normal([number_of_hidden_nodes, num_labels]))
    biases_softmax = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    relu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input_to_hidden) + biases_hidden)
    weights_hidden_to_softmax_drop = tf.nn.dropout(weights_hidden_to_softmax, keep_prob)
    logits = tf.matmul(relu,weights_hidden_to_softmax_drop) + biases_softmax
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_relu = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input_to_hidden) + biases_hidden)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_relu,weights_hidden_to_softmax) + biases_softmax)
    test_relu = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input_to_hidden) + biases_hidden)
    test_prediction = tf.nn.softmax(tf.matmul(test_relu,weights_hidden_to_softmax) + biases_softmax)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        drop_prob = 0.5
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : drop_prob}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 399.775421
Minibatch accuracy: 11.7%
Validation accuracy: 31.9%
Minibatch loss at step 500: 162.845779
Minibatch accuracy: 82.0%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 45.366852
Minibatch accuracy: 75.0%
Validation accuracy: 79.2%
Minibatch loss at step 1500: 36.502232
Minibatch accuracy: 77.3%
Validation accuracy: 75.5%
Minibatch loss at step 2000: 27.795084
Minibatch accuracy: 72.7%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 6.778937
Minibatch accuracy: 74.2%
Validation accuracy: 79.4%
Minibatch loss at step 3000: 5.318855
Minibatch accuracy: 76.6%
Validation accuracy: 78.2%
Test accuracy: 86.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [66]:
image_size = 28
num_labels = 10
train_subset = 10000
number_of_hidden1_nodes = 2048
number_of_hidden2_nodes = 1024
gamma = 0.01

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights_input_to_hidden1 = tf.Variable(tf.truncated_normal([image_size * image_size, number_of_hidden1_nodes]))
    biases_hidden1 = tf.Variable(tf.zeros([number_of_hidden1_nodes]))
    weights_hidden1_to_hidden2 = tf.Variable(tf.truncated_normal([number_of_hidden1_nodes, number_of_hidden2_nodes]))
    biases_hidden2 = tf.Variable(tf.zeros([number_of_hidden2_nodes]))
    weights_hidden2_to_softmax = tf.Variable(tf.truncated_normal([number_of_hidden2_nodes, num_labels]))
    biases_softmax = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    crelu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input_to_hidden1) + biases_hidden1)
    relu = tf.nn.relu(tf.matmul(crelu, weights_hidden1_to_hidden2) + biases_hidden2)
    #weights_hidden_to_softmax_drop = tf.nn.dropout(weights_hidden_to_softmax, keep_prob)
    logits = tf.matmul(relu, weights_hidden2_to_softmax) + biases_softmax
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+\
                            gamma*tf.reduce_mean(tf.nn.l2_loss(weights_input_to_hidden1))+\
                            gamma*tf.reduce_mean(tf.nn.l2_loss(weights_hidden1_to_hidden2))+\
                            gamma*tf.reduce_mean(tf.nn.l2_loss(weights_hidden2_to_softmax))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    crelu_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input_to_hidden1) + biases_hidden1)
    relu_valid = tf.nn.relu(tf.matmul(crelu_valid, weights_hidden1_to_hidden2) + biases_hidden2)
    valid_prediction = tf.nn.softmax(tf.matmul(relu_valid, weights_hidden2_to_softmax) + biases_softmax)
    
    crelu_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights_input_to_hidden1) + biases_hidden1)
    relu_test = tf.nn.relu(tf.matmul(crelu_test, weights_hidden1_to_hidden2) + biases_hidden2)
    test_prediction = tf.nn.softmax(tf.matmul(relu_test, weights_hidden2_to_softmax) + biases_softmax)

In [67]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 23648.064453
Training accuracy: 10.6%
Validation accuracy: 10.0%
Loss at step 100: nan
Training accuracy: 9.7%
Validation accuracy: 10.0%


KeyboardInterrupt: 

In [98]:
image_size = 28
num_labels = 10
train_subset = 10000
batch_size = 128
number_of_hidden1_nodes = 1024
number_of_hidden2_nodes = 2048
gamma = 0.0004

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    #tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    #tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights_input_to_hidden1 = tf.Variable(tf.truncated_normal([image_size * image_size, number_of_hidden1_nodes]))
    biases_hidden1 = tf.Variable(tf.zeros([number_of_hidden1_nodes]))
    weights_hidden1_to_hidden2 = tf.Variable(tf.truncated_normal([number_of_hidden1_nodes, number_of_hidden2_nodes]))
    biases_hidden2 = tf.Variable(tf.zeros([number_of_hidden2_nodes]))
    weights_hidden2_to_softmax = tf.Variable(tf.truncated_normal([number_of_hidden2_nodes, num_labels]))
    biases_softmax = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    crelu = tf.sigmoid(tf.matmul(tf_train_dataset, weights_input_to_hidden1) + biases_hidden1)
    relu = tf.nn.relu(tf.matmul(crelu, weights_hidden1_to_hidden2) + biases_hidden2)
    #weights_hidden_to_softmax_drop = tf.nn.dropout(weights_hidden_to_softmax, keep_prob)
    logits = tf.matmul(relu, weights_hidden2_to_softmax) + biases_softmax
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))+\
                            gamma*tf.reduce_mean(tf.nn.l2_loss(weights_input_to_hidden1))+\
                            gamma*tf.reduce_mean(tf.nn.l2_loss(weights_hidden1_to_hidden2))+\
                            gamma*tf.reduce_mean(tf.nn.l2_loss(weights_hidden2_to_softmax))
  
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    crelu_valid = tf.sigmoid(tf.matmul(tf_valid_dataset, weights_input_to_hidden1) + biases_hidden1)
    relu_valid = tf.nn.relu(tf.matmul(crelu_valid, weights_hidden1_to_hidden2) + biases_hidden2)
    valid_prediction = tf.nn.softmax(tf.matmul(relu_valid, weights_hidden2_to_softmax) + biases_softmax)
    
    crelu_test = tf.sigmoid(tf.matmul(tf_test_dataset, weights_input_to_hidden1) + biases_hidden1)
    relu_test = tf.nn.relu(tf.matmul(crelu_test, weights_hidden1_to_hidden2) + biases_hidden2)
    test_prediction = tf.nn.softmax(tf.matmul(relu_test, weights_hidden2_to_softmax) + biases_softmax)

In [99]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        drop_prob = 0.5
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 1285.933105
Training accuracy: 7.0%
Validation accuracy: 21.3%
Loss at step 100: 476.266235
Training accuracy: 68.0%
Validation accuracy: 67.0%
Loss at step 200: 457.070557
Training accuracy: 68.0%
Validation accuracy: 67.1%
Loss at step 300: 438.533936
Training accuracy: 62.5%
Validation accuracy: 67.6%
Loss at step 400: 421.167755
Training accuracy: 63.3%
Validation accuracy: 68.1%
Loss at step 500: 403.951752
Training accuracy: 76.6%
Validation accuracy: 69.7%
Loss at step 600: 389.041077
Training accuracy: 68.0%
Validation accuracy: 69.8%
Loss at step 700: 373.493713
Training accuracy: 71.9%
Validation accuracy: 70.6%
Loss at step 800: 358.532562
Training accuracy: 77.3%
Validation accuracy: 71.5%
Loss at step 900: 344.616333
Training accuracy: 69.5%
Validation accuracy: 72.9%
Loss at step 1000: 331.037170
Training accuracy: 76.6%
Validation accuracy: 73.5%
Loss at step 1100: 317.904388
Training accuracy: 77.3%
Validation accuracy: 74.4%
Loss at step 120